# Import các thư viện cần thiết

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
OPEN_API_KEY=os.getenv('OPEN_API_KEY')
COHERE_API_KEY=os.getenv('COHERE_API_KEY')
GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')
GROQ_API_kEY=os.getenv('GROQ_API_KEY')

In [1]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import ObsidianLoader
from langchain.document_loaders import PDFMinerLoader
from langchain.document_loaders import PDFPlumberLoader
from langchain.document_loaders import PyPDFium2Loader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders import PyMuPDFLoader

In [2]:
from langchain_community.document_loaders import JSONLoader,TextLoader

In [9]:
loader = TextLoader(r"./example_data/demo1.json")

In [10]:
import json

In [11]:
def process_json_file(file_path):
    json_data = []
    with open(file_path, 'r') as file:
        for line in file:
            try:
                data = json.loads(line)
                json_data.append(data)
            except json.JSONDecodeError:
                try:
                    data = json.loads(line[:-1])
                    json_data.append(data)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
    return json_data

In [12]:
texts = process_json_file(r"./example_data/demo1.json")

Error decoding JSON: Expecting value: line 1 column 2 (char 1)
Error decoding JSON: Expecting property name enclosed in double quotes: line 1 column 6 (char 5)
Error decoding JSON: Extra data: line 1 column 21 (char 20)
Error decoding JSON: Extra data: line 1 column 19 (char 18)
Error decoding JSON: Extra data: line 1 column 29 (char 28)
Error decoding JSON: Extra data: line 1 column 23 (char 22)
Error decoding JSON: Extra data: line 1 column 23 (char 22)
Error decoding JSON: Extra data: line 1 column 24 (char 23)
Error decoding JSON: Expecting value: line 1 column 13 (char 12)
Error decoding JSON: Extra data: line 1 column 28 (char 27)
Error decoding JSON: Extra data: line 1 column 28 (char 27)
Error decoding JSON: Extra data: line 1 column 26 (char 25)
Error decoding JSON: Expecting value: line 1 column 9 (char 8)
Error decoding JSON: Extra data: line 1 column 15 (char 14)
Error decoding JSON: Extra data: line 1 column 15 (char 14)
Error decoding JSON: Expecting value: line 1 column 

In [13]:
check = loader.load()

In [14]:
print(check)

[Document(metadata={'source': './example_data/demo1.json'}, page_content='[\n    {\n        "element_id": "5b6caa33b8d30dd2475548f31b167dfe",\n        "metadata": {\n            "file_directory": "D:\\\\KLTN\\\\Document_Loader\\\\docs",\n            "filename": "demo1.pdf",\n            "filetype": "application/pdf",\n            "languages": [\n                "eng"\n            ],\n            "last_modified": "2024-04-13T00:19:19",\n            "orig_elements": "eJzNVl2L4zYU/SsXP6+MvizLoX3Ydcs4bONJGyd0mR0GWZITw8QZJg7ddNn/Xn0k01BSui9DBwKRzr1X9+ocnZC7r4l9tFs7jA+9SSaQZLnuaEE6RETHES8yjSRvFbKc55oJ0VKjkneQbO2ojBqVq/ma6N3u2fSDGu0+7B/VcXcYHza2X29Gh1CKsas5wX/0Ztw4lOQBfdr1w+jr7u4IzlIH8SzN79/BeUuxJCkLgOAyZdeQUOKAZH/cj3brbzLvv9jHxZPSNvnmAl3/aMfjkw2h+SwJ8wzrg1qHoe8SO6yT+4Dux4ftzvRdbwMlFFOOMEeENRhPSOE+vvrJVT4Mh21rn/1lfI/RfvHXTT58PhBrJNxMPx8w1uQWfvKI5SWsAoLBfRGC4+YWGh9WbgHNbz6iunCCrm9eEn18CpVfaF3CIqbBvIqhGXz0i05CUy0DpBto5mlVzgA+VrfvofTNDHdH1jfxGOFyX8BmWrvu0xCgS/jldEgNZbX8FOZUtauME9cVXAzmrtNEfDo5rf

In [19]:
import os
import shutil
from dotenv import load_dotenv
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredFileIOLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader
from langchain_community.document_loaders import UnstructuredCSVLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import UnstructuredXMLLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
import os
import google.generativeai as genai
import re
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere
import pandas as pd
import json


# Xây dựng các hàm sử dụng

In [20]:

def find_matching_files_in_docs_12_id(text,id):
    folder_path = f"./code/temp/{id}"
    search_terms = []
    search_terms_old = []
    matching_index = []    
    search_origin = re.findall(r'\b\w+\.\w+\b|\b\w+\b', text)
    search_terms_origin = []
    for word in search_origin:
       if '.' in word:
        search_terms_origin.append(word)
       else:
        search_terms_origin.extend(re.findall(r'\b\w+\b', word))
    file_names_with_extension = re.findall(r'\b\w+\.\w+\b|\b\w+\b', text.lower())
    file_names_with_extension_old = re.findall(r'\b(\w+\.\w+)\b', text)
    for file_name in search_terms_origin:
        if "." in file_name:
            term_position = search_terms_origin.index(file_name)
            search_terms_old.append(file_name)
    for file_name in file_names_with_extension_old:
        if "." in file_name:
            search_terms_old.append(file_name)
    for file_name in file_names_with_extension:
            search_terms.append(file_name)
    clean_text_old = text
    clean_text = text.lower()
    for term in search_terms_old:
        clean_text_old = clean_text_old.replace(term, '')
    for term in search_terms:
        clean_text = clean_text.replace(term, '')
    words = re.findall(r'\b\w+\.\w+\b|\b\w+\b', text)

    words_old = re.findall(r'\b\w+\b', clean_text_old)
    search_terms_old.extend(words_old)

    matching_files = set()
    matching_files_old = set()
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            for term in search_terms:
                if term.lower() in file.lower():
                    term_position = search_terms.index(term)
                    term_value = search_terms_origin[term_position]
                    matching_files.add(file)
                    matching_index.append(term_position)
                    break
    matching_files_old1 = []
    matching_index.sort()
    for x in matching_index:
        matching_files_old1.append(search_terms_origin[x])
    
    return matching_files,matching_files_old1

In [22]:
def extract_data2(id):
 documents = []
 for file in os.listdir(f"/code/temp/{id}"):
    if file.endswith(".pdf"):
        pdf_path = f"/code/temp/{id}/" + file
        loader = UnstructuredPDFLoader(pdf_path)
        documents.extend(loader.load())
    elif file.endswith('.docx') or file.endswith('.doc'):
        doc_path = f"/code/temp/{id}/" + file
        loader = Docx2txtLoader(doc_path)
        documents.extend(loader.load())
    elif file.endswith('.txt'):
        txt_path = f"/code/temp/{id}/" + file
        loader = TextLoader(txt_path,encoding="utf8")
        documents.extend(loader.load())
    elif file.endswith('.pptx'):
        ppt_path = f"/code/temp/{id}/" + file
        loader = UnstructuredPowerPointLoader(ppt_path)
        documents.extend(loader.load())
    elif file.endswith('.csv'):
        csv_path = f"/code/temp/{id}/" + file
        loader = UnstructuredCSVLoader(csv_path)
        documents.extend(loader.load())
    elif file.endswith('.xlsx'):
        excel_path = f"/code/temp/{id}/" + file
        loader = UnstructuredExcelLoader(excel_path)
        documents.extend(loader.load())
    elif file.endswith('.xml'):
        xml_path = f"/code/temp/{id}/" + file
        loader = UnstructuredXMLLoader(xml_path)
        documents.extend(loader.load())
    elif file.endswith('.html'):
        html_path = f"/code/temp/{id}/" + file
        loader = UnstructuredHTMLLoader(html_path)
        documents.extend(loader.load())
    elif file.endswith('.json'):
        json_path = f"/code/temp/{id}/" + file
        loader = TextLoader(json_path)
        documents.extend(loader.load())
    elif file.endswith('.md'):
        json_path = f"/code/temp/{id}/" + file
        loader = UnstructuredMarkdownLoader(json_path)
        documents.extend(loader.load())  
 text_splitter = CharacterTextSplitter(chunk_size=4500, chunk_overlap=2500)
 texts= text_splitter.split_documents(documents)
 text_all = texts
 return text_all

In [23]:
def convert_xlsx_to_csv(xlsx_file_path, csv_file_path):
    df = pd.read_excel(xlsx_file_path)
    df.to_csv(csv_file_path, index=False)

In [24]:
def save_list_CSV(file_list):
    text = ""
    for x in file_list:
        if x.endswith('.xlsx'):
            old = f"/code/temp/{x}"
            new = old.replace(".xlsx", ".csv")
            convert_xlsx_to_csv(old, new)
            x = x.replace(".xlsx", ".csv")
        loader1 = CSVLoader(f"/code/temp/{x}") 
        print(x)
        docs1 = loader1.load()
        text += f"Dữ liệu file {x}:\n"
        for z in docs1:
            text += z.page_content + "\n"
        
    return text

In [25]:
def extract_content_between_keywords(query, keywords):
    contents = {}
    num_keywords = len(keywords)
    keyword_positions = []
    
    for i in range(num_keywords):
        keyword = keywords[i]
        keyword_position = query.find(keyword)
        keyword_positions.append(keyword_position)
        
        if keyword_position == -1:
            continue
        
        next_keyword_position = len(query)
        for j in range(i + 1, num_keywords):
            next_keyword = keywords[j]
            next_keyword_position = query.find(next_keyword)
            if next_keyword_position != -1:
                break
        if i == 0:
            content_before = query[:keyword_position].strip()
        else:
            content_before = query[keyword_positions[i-1] + len(keywords[i-1]):keyword_position].strip()
        if i == num_keywords - 1:
            content_after = query[keyword_position + len(keyword):].strip()
        else:
            content_after = query[keyword_position + len(keyword):next_keyword_position].strip()
        content = f"{content_before} {keyword} {content_after}"
        contents[keyword] = content
    
    return contents

In [26]:
def merge_files(file_set, file_list):
    """Hàm này ghép lại các tên file dựa trên điều kiện đã cho."""
    merged_files = {}
    
    for file_name in file_list:
        name = file_name.split('.')[0]
        for f in file_set:
            if name in f:
                merged_files[name] = f
                break
        
    return merged_files

In [27]:
def replace_keys_with_values(original_dict, replacement_dict):
    """
    Thay thế các key trong original_dict bằng các giá trị tương ứng từ replacement_dict.
    
    Tham số:
        - original_dict: Từ điển gốc cần thay đổi.
        - replacement_dict: Từ điển chứa các cặp key-value sẽ được sử dụng để thay thế key trong original_dict.
    
    Trả về:
        - new_dict: Từ điển mới sau khi thực hiện thay đổi.
    """
    new_dict = {}
    for key, value in original_dict.items():
        if key in replacement_dict:
            new_key = replacement_dict[key]
            new_dict[new_key] = value
        else:
            new_dict[key] = value
    return new_dict

def aws1_csv(new_dict_csv):
 text = ""
 query_all = ""
 for key, value in new_dict_csv.items():
    print(key,value)
    query = value
    query_all += value
    keyword = []
    keyword.append(key)
    print(keyword)
    test = save_list_CSV(keyword)
    text += test
 return text,query_all

In [28]:
from groq import Groq

client = Groq(
    api_key=GROQ_API_kEY,
)

os.environ["COHERE_API_KEY"] = COHERE_API_KEY
genai.configure(api_key=GOOGLE_API_KEY)
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [29]:
def get_chat_completion(prompt_query):
 try:
  chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "Bạn là một trợ lý trung thưc, trả lời dựa trên nội dung tài liệu được cung cấp. Chỉ trả lời liên quan đến câu hỏi một cách đầy đủ chính xác, không bỏ sót thông tin."
        },
        {
            "role": "user",
            "content": f"{prompt_query}",
        }
    ],

    # The language model which will generate the completion.
    model="llama3-70b-8192",
    temperature=0.0,
    # The maximum number of tokens to generate. Requests can use up to
    # 2048 tokens shared between prompt and completion.
    max_tokens=9000,

    # Controls diversity via nucleus sampling: 0.5 means half of all
    # likelihood-weighted options are considered.
    #top_p=1,

    # A stop sequence is a predefined or user-specified text string that
    # signals an AI to stop generating content, ensuring its responses
    # remain focused and concise. Examples include punctuation marks and
    # markers like "[end]".
    stop=None,

    # If set, partial message deltas will be sent.
    stream=False,
  )
  return chat_completion.choices[0].message.content
 except Exception as error:
    # Handle the RateLimitError here
    #print("Rate limit reached. Please try again later.")
    #print("Error message:", error.message)
    return False
# Print the completion returned by the LLM.

def initialize_generative_model(prompt):
    # Set up the model generation configuration
    generation_config = {
        "temperature": 0.0,
        "top_p": 0.0,
        "top_k": 0,
        "max_output_tokens": 8192,
    }

    # Define safety settings
    safety_settings = [
        {
            "category": "HARM_CATEGORY_HARASSMENT",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE"
        },
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE"
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE"
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_MEDIUM_AND_ABOVE"
        },
    ]

    # Initialize the generative model
    model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                                  generation_config=generation_config,
                                  safety_settings=safety_settings)
    convo = model.start_chat(history=[])
    convo.send_message(prompt)
    return convo.last.text


In [30]:
def question_answer(question):
    completion = get_chat_completion(question)
    if completion:
        return completion
    else:
        answer = initialize_generative_model(question)
        return answer

In [31]:
def extract_multi_metadata_content(texts, tests):
    extracted_content = ""
    for idx, test in enumerate(tests):
        temp_content = ""
        for x in texts:
            metadata_lower = x.metadata['source'].lower()
            if any(term.lower() in metadata_lower for term in test.split()): 
                temp_content += x.page_content
        if idx == 0:
            extracted_content += f"Dữ liệu của {test}:\n {temp_content}"
        else:
            extracted_content += "\n" + temp_content + "\n"
    return extracted_content

In [32]:
def aws1_all(new_dict,text_alls):
 answer = ""
 for key, value in new_dict.items():
    query = value
    keyword,keyword2= find_matching_files_in_docs_12_id(query,id)
    print(value)
    print(keyword)
    data= extract_multi_metadata_content(text_alls,keyword)
    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=2200, chunk_overlap=400)
    texts_data = text_splitter.split_text(data)
    persist_directory = f'{key}'
    vectordb = Chroma.from_texts(texts_data,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)
    k_1 = round(len(texts_data))
    
    retriever = vectordb.as_retriever(search_kwargs={f"k":k_1})
    llm = Cohere(temperature=0)
    compressor = CohereRerank()
    compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
    )

    compressed_docs = compression_retriever.get_relevant_documents(
    f"{query}"
    )
    text = ""
    text += "Dữ liệu file" + f"{key}"
    text = ''.join([x1.page_content for x1 in compressed_docs])

    
    prompt_document = f"Dựa vào nội dung sau:{text}. Hãy trả lời câu hỏi sau đây: {query}. Mà không thay đổi, chỉnh sửa nội dung mà mình đã cung cấp"
    answer_for = question_answer(prompt_document)
    answer += answer_for + "\n"

    
 return answer

# Xây dựng hệ thống xử lý

### Bước 1: Indexing

#### Áp dụng chunk phù hợp

In [33]:
def extract_data():
 documents = []
 for file in os.listdir("./code/temp"):
    if file.endswith(".pdf"):
        pdf_path = "./code/temp/" + file
        loader = UnstructuredPDFLoader(pdf_path)
        documents.extend(loader.load())
    elif file.endswith('.docx') or file.endswith('.doc'):
        doc_path = "./code/temp/" + file
        loader = Docx2txtLoader(doc_path)
        documents.extend(loader.load())
    elif file.endswith('.txt'):
        txt_path = "./code/temp/" + file
        loader = TextLoader(txt_path,encoding="utf8")
        documents.extend(loader.load())
    elif file.endswith('.pptx'):
        ppt_path = "./code/temp/" + file
        loader = UnstructuredPowerPointLoader(ppt_path)
        documents.extend(loader.load())
    elif file.endswith('.csv'):
        csv_path = "./code/temp/" + file
        loader = UnstructuredCSVLoader(csv_path)
        documents.extend(loader.load())
    elif file.endswith('.xlsx'):
        excel_path = "./code/temp/" + file
        loader = UnstructuredExcelLoader(excel_path)
        documents.extend(loader.load())
    elif file.endswith('.xml'):
        xml_path = "./code/temp/" + file
        loader = UnstructuredXMLLoader(xml_path)
        documents.extend(loader.load())
    elif file.endswith('.html'):
        html_path = "./code/temp/" + file
        loader = UnstructuredHTMLLoader(html_path)
        documents.extend(loader.load())
    elif file.endswith('.json'):
        json_path = "./code/temp/" + file
        loader = JSONLoader(json_path)
        documents.extend(loader.load())
    elif file.endswith('.md'):
        json_path = "./code/temp/" + file
        loader = UnstructuredMarkdownLoader(json_path)
        documents.extend(loader.load())  
 #Phân chia dữ liệu
 text_splitter = CharacterTextSplitter(chunk_size=2200, chunk_overlap=1500)
 texts= text_splitter.split_documents(documents)
 text_all = texts

 return text_all


In [34]:

text_all = extract_data()

Created a chunk of size 3869, which is longer than the specified 2200
Created a chunk of size 3964, which is longer than the specified 2200
Created a chunk of size 3542, which is longer than the specified 2200


- Qua quá trình thử nghiệm nhóm nhận thấy rằng sử dụng mức chunk 4500 và overlap 2500 là mức chunk vừa phải để có thể lấy hết nội dung dữ liệu của file chỉ chứa 1 tập pdf.
- Sử dụng mức chunk này sẽ chia dữ liệu với mỗi chunk có thể tầm với 1500 tương đương 700 token thì 4500 tầm 2100token --> Nếu dùng cho các bước sau sẽ là số token hoàn chỉnh để đẩy vào LLM

In [35]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size = 4500,chunk_overlap = 2500)
texts = text_splitter.split_documents(text_all)

In [36]:
print(len(texts))

161


In [37]:
print(text_all)

[Document(metadata={'source': './code/temp/CV_VoNhuY_Java.pdf'}, page_content='SOCIAL\n\n✉ 20133118@student.hcmute.edu.vn\n\n\ue11b 0769674810\n\n\uf0ac https://www.facebook.com/\n\nvonhuy15112002\n\n\uf3c5 99 Street 7, Linh Trung, Thu Duc City,\n\nHo Chi Minh\n\n\ue0d5 https://github.com/ohhhchank3/\n\nSKILLS\n\nPROGAMMING\n\nPython • Java • C#\n\nFRAMEWORK • Spring Boot • Spring MVC • Bootstrap\n\nDATABASE\n\nSQL Server • MySQL\n\nVERSION CONTROL\n\nGit • GitLab, GitHub\n\nHOBBIES\n\nMusic • Sports\n\nComunicate • Read Books\n\nVO NHU Y\n\nJAVA DEVELOPER - JAVA INTERN\n\nCAREER GOALS\n\nShort-term goal (within the next 2 years): Dedicate e\x00orts to learning and evolving into a Full Stack Developer, with the speci\x00c objective of achieving OCP certi\x00cation.\n\nLong-term goal (from 3 to 5 years ahead): Actualize the aspiration to enhance professional expertise, strengthen knowledge, and progress towards a role as a\n\nSenior Java Developer or, potentially, a Tech Lead Java. Addi

### Bước 2: Query Transformation 

- Ở đây nhóm tiến hành sử dụng một mô hình của OpenAI để có thể thực hiện Multi-Step Query tức là chia câu query đa dạng thành một câu query truy vấn đơn giản và dễ sử dụng

In [38]:
from typing import List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Search(BaseModel):

    queries: List[str] = Field(
        ...,
        description="Truy vấn riêng biệt để tìm kiếm, giữ nguyên ý chính câu hỏi riêng biệt",
    )

In [39]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

In [40]:
import os

In [42]:
os.environ["OPENAI_API_KEY"] = OPEN_API_KEY

In [43]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

output_parser = PydanticToolsParser(tools=[Search])

system = """Bạn có khả năng đưa ra các truy vấn tìm kiếm chính xác để lấy thông tin giúp trả lời các yêu cầu của người dùng. Các truy vấn của bạn phải chính xác, không được bỏ ngắn rút gọn.

Nếu bạn cần tra cứu hai hoặc nhiều thông tin riêng biệt, bạn có thể làm điều đó!. Trả lời câu hỏi bằng tiếng Việt(Vietnamese), không được dùng ngôn ngữ khác"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.0)
structured_llm = llm.with_structured_output(Search)
query_analyzer = {"question": RunnablePassthrough()} | prompt | structured_llm

- Hãy thử nghiệm với câu query dưới đây để xem sự phân chia câu hỏi

In [44]:
test = query_analyzer.invoke("hãy cho biết tên sinh viên thực hiện đề tài trong demo1.pdf và hãy cho biết số điện thoại trong CV_VoNhuY_Java.pdf ; hãy cho biết MSSV của Đặng Xuân Bách; hãy cho biết tên giáo viên phản biện của sinh viên Võ Như Ý; hãy cho biết MSSV của Nguyễn Quang Phúc")

In [45]:
print(test)

queries=['Tên sinh viên thực hiện đề tài trong demo1.pdf', 'Số điện thoại trong CV_VoNhuY_Java.pdf', 'MSSV của Đặng Xuân Bách', 'Tên giáo viên phản biện của sinh viên Võ Như Ý', 'MSSV của Nguyễn Quang Phúc']


#### Áp dụng 2 kỹ thuật Logical Routing và Semantic Routing:

In [47]:
import re
#Phân tách query ra từng câu riêng lẻ.
test_string = str(test)
matches = re.findall(r"'([^']*)'", test_string)

In [48]:
for x in matches:
    print(x)

Tên sinh viên thực hiện đề tài trong demo1.pdf
Số điện thoại trong CV_VoNhuY_Java.pdf
MSSV của Đặng Xuân Bách
Tên giáo viên phản biện của sinh viên Võ Như Ý
MSSV của Nguyễn Quang Phúc


##### Áp dụng kỹ thuật Logical Routing

- Ở đây nhóm đã phát triển kĩ thuật này bằng cách sử dụng hàm find_matching_files_in_docs_12:thì cơ bản đây là môt hàm nhận vào một câu query và id để chứa thư mục người dùng. Sẽ tiến hành tiềm kiếm trong câu query có các từ khóa keyword hay không nếu có từ khóa đó hãy chuyển hướng nó thẳng sang đối tượng là file đó và xử lý. Các keyword nhận dạng được sẽ là tên file.pdf tên file một phần hoặc toàn bộ tên file

* Hãy thử nghiệm xem kết quả

In [49]:
def find_matching_files_in_docs_12(text):
    folder_path = "./code/temp"
    search_terms = []
    search_terms_old = []
    matching_index = []

    search_origin = re.findall(r'\b\w+\.\w+\b|\b\w+\b', text)

    search_terms_origin = []
    for word in search_origin:
       if '.' in word:
        search_terms_origin.append(word)
       else:
        search_terms_origin.extend(re.findall(r'\b\w+\b', word))

    file_names_with_extension = re.findall(r'\b\w+\.\w+\b|\b\w+\b', text.lower())
    file_names_with_extension_old = re.findall(r'\b(\w+\.\w+)\b', text)
    for file_name in search_terms_origin:
        if "." in file_name:
            term_position = search_terms_origin.index(file_name)
            search_terms_old.append(file_name)
    for file_name in file_names_with_extension_old:
        if "." in file_name:
            search_terms_old.append(file_name)
    for file_name in file_names_with_extension:

            search_terms.append(file_name)
    clean_text_old = text
    clean_text = text.lower()
    for term in search_terms_old:
        clean_text_old = clean_text_old.replace(term, '')
    for term in search_terms:
        clean_text = clean_text.replace(term, '')

    words = re.findall(r'\b\w+\.\w+\b|\b\w+\b', text)

    words_old = re.findall(r'\b\w+\b', clean_text_old)
    search_terms_old.extend(words_old)
    matching_files = set()
    matching_files_old = set()
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            for term in search_terms:
                if term.lower() in file.lower():
                    term_position = search_terms.index(term)
                    term_value = search_terms_origin[term_position]
                    matching_files.add(file)
                    matching_index.append(term_position)
                    break
    matching_files_old1 = []
    matching_index.sort()
    for x in matching_index:
        matching_files_old1.append(search_terms_origin[x])
    
    return matching_files,matching_files_old1

In [50]:
# Sẽ sử dụng 1 câu hỏi: Tên sinh viên thực hiện đề tài trong demo1.pdf
keyword,key_words_old = find_matching_files_in_docs_12("Tên sinh viên thực hiện đề tài trong demo1.pdf")
file_list = keyword


In [62]:
print(keyword)
print(key_words_old)

{'demo1.pdf'}
['demo1.pdf']


- Như vậy ở đây có thể thấy rằng đã tìm thấy thư mục keyword --> xác định bước xử lý tiếp theo trong phần 3 phần 4

##### Áp dụng kỹ thuật Semantic Routing

- Với phương pháp này nhóm dùng ngữ nghĩa câu hỏi kết hợp rerank và tìm kết quả trả về cao liên quan nhất và gửi đến định hướng đến file liên quan đó và xử lý 

In [51]:
# Import thư viện
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere

In [52]:
# Định nghĩa Cohere
import os

os.environ["COHERE_API_KEY"] = "7zxb6XL0maslJyRmAZ0QDbHEffsVKvFwwzMEs3Bc"

In [53]:
# Bây h hãy lưu trữ database và thiết lập một retriever ở đây chỉ trình bày cách tiếp cận không đụng tới Phần 3
vectordb2 = Chroma.from_documents(texts,
                                 embedding=embeddings,persist_directory="./demo")


In [54]:
k = len(texts)

In [55]:
# Thiết lập số k cần truy vấn cho vectordb
retriever = vectordb2.as_retriever(search_kwargs={f"k":k})

In [56]:
compressor_file = CohereRerank(top_n=5,model="rerank-english-v2.0")
compression_retriever_file = ContextualCompressionRetriever(
        base_compressor=compressor_file, base_retriever=retriever
            )
compressed_docs_file = compression_retriever_file.get_relevant_documents("Hãy cho biết MSSV Của Võ Như Ý") 

d:\Set_Up\Code_ipynb\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [57]:
relevance_score_float = float(compressed_docs_file[0].metadata['relevance_score'])

In [58]:
file_path = compressed_docs_file[0].metadata['source']

In [59]:
print(file_path)

./code/temp/demo1.pdf


- Ở đây nhóm đã tiềm ra đoạn liên quan tới file demo1.pdf và chuyển hướng tiếp cận của mình tới file này để xử lý câu hỏi

### Bước 3: Retrieval

#### Fusion Retriever

- Nhóm tiến hành áp dụng kỹ thuật Fusion Retriever để có thể tối ưu truy vấn từ nhiều nguồn khác nhau

In [60]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document

from langchain.vectorstores import FAISS


In [61]:
k = len(texts)

In [63]:
# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 10

In [64]:
bm25_retriever.get_relevant_documents("Hãy cho biết họ tên giáo viên phản biện trong demo1.pdf")

[Document(metadata={'source': './code/temp/demo1.pdf'}, page_content='…………….………..……….……………………………………………………………… ………………………………………………………………………………………………\n\n3. Đề nghị cho bảo vệ hay không ?..........................................................................................\n\n4. Đánh giá loại : ………………………………………………………………………………\n\n5. Điểm : ………………………………………………………………………………………\n\nTp. Hồ Chí Minh, ngày 22 tháng 12 năm 2023\n\nGiáo viên hướng dẫn\n\n(Ký & ghi rõ họ tên)\n\nĐH SƯ PHẠM KỸ THUẬT TP.HCM KHOA CNTT Độc *******\n\nCỘNG HOÀ\n\nXÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập – Tự do – Hạnh Phúc *******\n\nPHIÊU NHẬN XÉT CỦA GIÁO VIÊN PHẢN BIỆN\n\nHọ va tên Sinh viên 1 : Võ Như Ý\n\nMSSV 1: 20133118\n\nHọ va tên Sinh viên 2 : Nguyễn Quang Phúc\n\nMSSV 2: 20133080\n\nNganh: Kỹ thuật dữ liệu\n\nTên đề tai: Tìm hiểu về các thư viện hỗ trợ xây dựng chatbot\n\nHọ va tên Giáo viên phản biện: TS. Nguyễn Thanh Sơn\n\nNHẬN XÉT Về nội dung đề tai & khối lượng thực hiện: …………….………..……….……………………………………………………………..

In [65]:
bm25_retriever.dict

<bound method BaseModel.dict of BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x0000024865AA1210>, k=10)>

In [66]:
retriever = vectordb2.as_retriever(search_kwargs={f"k":10})

In [67]:
retriever.get_relevant_documents("Hãy cho biết họ tên giáo viên phản biện trong demo1.pdf")

[Document(metadata={'source': './code/temp/demo1.pdf'}, page_content='MSSV 1: 20133118\n\nHọ va tên Sinh viên 2 : Nguyễn Quang Phúc\n\nMSSV 2: 20133080\n\nNganh: Kỹ thuật dữ liệu\n\nTên đề tai: Tìm hiểu về các thư viện hỗ trợ xây dựng chatbot\n\nHọ va tên Giáo viên phản biện: TS. Nguyễn Thanh Sơn\n\nNHẬN XÉT Về nội dung đề tai & khối lượng thực hiện: …………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n1. Ưu điểm:\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….…………………………………………………………….. …………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n2. Khuyết điểm\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n…

In [68]:
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever],
                                       weights=[0.8, 0.2])

- Sử dụng Ensemble để kết hợp kết quả từ truy vấn

In [69]:
docs = ensemble_retriever.get_relevant_documents("Hãy cho biết họ tên giáo viên phản biện trong demo1.pdf")
docs

[Document(metadata={'source': './code/temp/demo1.pdf'}, page_content='MSSV 1: 20133118\n\nHọ va tên Sinh viên 2 : Nguyễn Quang Phúc\n\nMSSV 2: 20133080\n\nNganh: Kỹ thuật dữ liệu\n\nTên đề tai: Tìm hiểu về các thư viện hỗ trợ xây dựng chatbot\n\nHọ va tên Giáo viên phản biện: TS. Nguyễn Thanh Sơn\n\nNHẬN XÉT Về nội dung đề tai & khối lượng thực hiện: …………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n1. Ưu điểm:\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….…………………………………………………………….. …………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n2. Khuyết điểm\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n…

In [70]:
docs[1].page_content

'…………….………..……….……………………………………………………………… ………………………………………………………………………………………………\n\n3. Đề nghị cho bảo vệ hay không ?..........................................................................................\n\n4. Đánh giá loại : ………………………………………………………………………………\n\n5. Điểm : ………………………………………………………………………………………\n\nTp. Hồ Chí Minh, ngày 22 tháng 12 năm 2023\n\nGiáo viên hướng dẫn\n\n(Ký & ghi rõ họ tên)\n\nĐH SƯ PHẠM KỸ THUẬT TP.HCM KHOA CNTT Độc *******\n\nCỘNG HOÀ\n\nXÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập – Tự do – Hạnh Phúc *******\n\nPHIÊU NHẬN XÉT CỦA GIÁO VIÊN PHẢN BIỆN\n\nHọ va tên Sinh viên 1 : Võ Như Ý\n\nMSSV 1: 20133118\n\nHọ va tên Sinh viên 2 : Nguyễn Quang Phúc\n\nMSSV 2: 20133080\n\nNganh: Kỹ thuật dữ liệu\n\nTên đề tai: Tìm hiểu về các thư viện hỗ trợ xây dựng chatbot\n\nHọ va tên Giáo viên phản biện: TS. Nguyễn Thanh Sơn\n\nNHẬN XÉT Về nội dung đề tai & khối lượng thực hiện: …………….………..……….……………………………………………………………..\n\n…………….………..……….……………………………………………………………..\n\n…………….………..……….………………

### Bước 4: Post - Retrieval

#### Kỹ thuật 1: Rerank với Cohere

In [71]:

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_community.llms import Cohere

In [72]:

vectordb3 = Chroma.from_documents(docs,
                                 embedding=embeddings)


In [73]:
print(len(docs))

12


In [74]:
retrieve3 = vectordb3.as_retriever(search_kwargs={f"k":13})

In [75]:
os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY_2")

In [76]:
# Thiết lập rerank cho câu hỏi query
llm = Cohere(temperature=0)
compressor = CohereRerank(top_n= 3,model="rerank-english-v2.0")
compression_retriever = ContextualCompressionRetriever(
  base_compressor=compressor, base_retriever= retriever
            )
compressed_docs2 = compression_retriever.get_relevant_documents(f"Hãy cho biết họ tên giáo viên phản biện trong demo1.pdf")

d:\Set_Up\Code_ipynb\venv\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  warn_deprecated(


In [77]:
print(compressed_docs2[0].page_content)

BỘ GIÁO DỤC VÀ ĐÀO TẠO

TRƯỜNG ĐẠI HỌC SƯ PHẠM KỸ THUẬT TP.HCM

KHOA CÔNG NGHỆ THÔNG TIN

TIỂU LUẬN CHUYÊN NGÀNH

ĐỀ TÀI: TÌM HIỂU VỀ CÁC THƯ VIỆN HỖ

TRỢ XÂY DỰNG CHATBOT

SINH VIÊN THỰC HIỆN: MSSV:20133118 Võ Như Ý Nguyễn Quang Phúc MSSV:20133080

GIÁO VIÊN HƯỚNG DẪN: Ts. Trần Nhật Quang

TP. Hồ Chí Minh - 2023

1

ĐH SƯ PHẠM KỸ THUẬT TP.HCM KHOA CNTT Độc *******

CỘNG HOÀ

XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập – Tự do – Hạnh Phúc *******

PHIÊU NHẬN XÉT CỦA GIÁO VIÊN HƯỚNG DẪN

Họ va tên Sinh viên 1 : Võ Như Ý

MSSV 1: 20133118

Họ va tên Sinh viên 2 : Nguyễn Quang Phúc

MSSV 2: 20133080

Nganh: Kỹ thuật dữ liệu

Tên đề tai: Tìm hiểu về các thư viện hỗ trợ xây dựng chatbot

Họ va tên Giáo viên hướng dẫn: TS. Trần Nhật Quang

NHẬN XÉT Về nội dung đề tai & khối lượng thực hiện: …………….………..……….……………………………………………………………..

…………….………..……….……………………………………………………………..

…………….………..……….……………………………………………………………..

…………….………..……….……………………………………………………………..

…………….………..……….……………………………………………………………..

…………

#### Kỹ thuật Compress + Filter

In [78]:
from langchain.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline

In [79]:
# Thiết lập model
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model='gemini-pro',
                    max_output_tokens=1024,
                    temperature=0.1,
                    top_p=0.2,
                    top_k=40,convert_system_message_to_human=True)

In [80]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

#creating the compressor
compressor = LLMChainExtractor.from_llm(llm=llm)

#compressor retriver = base retriever + compressor
compression_retriever = ContextualCompressionRetriever(base_retriever=retrieve3,
                                                       base_compressor=compressor)

In [81]:
from getpass import getpass
import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.document_compressors import EmbeddingsFilter

embdeddings_filter = EmbeddingsFilter(embeddings=embeddings)
compression_retriever_filter = ContextualCompressionRetriever(base_retriever=retrieve3,
                                                       base_compressor=embdeddings_filter)

compressed_docs = compression_retriever_filter.get_relevant_documents(query="Hãy cho biết họ tên giáo viên phản biện trong demo1.pdf")


Number of requested results 13 is greater than number of elements in index 12, updating n_results = 12


In [82]:
print(compressed_docs[0].page_content)

MSSV 1: 20133118

Họ va tên Sinh viên 2 : Nguyễn Quang Phúc

MSSV 2: 20133080

Nganh: Kỹ thuật dữ liệu

Tên đề tai: Tìm hiểu về các thư viện hỗ trợ xây dựng chatbot

Họ va tên Giáo viên phản biện: TS. Nguyễn Thanh Sơn

NHẬN XÉT Về nội dung đề tai & khối lượng thực hiện: …………….………..……….……………………………………………………………..

…………….………..……….……………………………………………………………..

…………….………..……….……………………………………………………………..

…………….………..……….……………………………………………………………..

…………….………..……….……………………………………………………………..

…………….………..……….……………………………………………………………..

1. Ưu điểm:

…………….………..……….……………………………………………………………..

…………….………..……….……………………………………………………………..

…………….………..……….……………………………………………………………..

…………….………..……….…………………………………………………………….. …………….………..……….……………………………………………………………..

…………….………..……….……………………………………………………………..

2. Khuyết điểm

…………….………..……….……………………………………………………………..

…………….………..……….……………………………………………………………..

…………….………..……….…………………………………………………………….. …………….………..……….……………………………………………………………. …………….………..……….……………………………………………

In [ ]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=compression_retriever_filter,
                                 verbose=True)
#Ask Question
qa("Hãy cho biết họ tên giáo viên phản biện trong demo1.pdf")

In [84]:
from langchain.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline

redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
relevant_filter = EmbeddingsFilter(embeddings=embeddings,k=5)

#creating the pipeline
compressed_pipeline = DocumentCompressorPipeline(transformers=[redundant_filter,relevant_filter])

# compressor retriever
comp_pipe_retrieve = ContextualCompressionRetriever(base_retriever=retrieve3,
                                                       base_compressor=compressed_pipeline)

# print the prompt
print(comp_pipe_retrieve)

# Get relevant documents
compressed_docs = comp_pipe_retrieve.get_relevant_documents(query="What is Environment?")


base_compressor=DocumentCompressorPipeline(transformers=[EmbeddingsRedundantFilter(embeddings=GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002483353BE90>, model='models/embedding-001', task_type=None, google_api_key=None, credentials=None, client_options=None, transport=None, request_options=None), similarity_fn=<function cosine_similarity at 0x000002482D174B80>, similarity_threshold=0.95), EmbeddingsFilter(embeddings=GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002483353BE90>, model='models/embedding-001', task_type=None, google_api_key=None, credentials=None, client_options=None, transport=None, request_options=None), similarity_fn=<function cosine_similarity at 0x000002482D174B80>, k=5, similarity_threshold=None)]) base_retriever=VectorStoreRetriever(tags=['Chroma', 'GoogleGenerativ

Number of requested results 13 is greater than number of elements in index 12, updating n_results = 12


In [85]:
print(compressed_docs[1].page_content)

…………….………..……….……………………………………………………………..

…………….………..……….…………………………………………………………….. …………….………..……….……………………………………………………………..

…………….………..……….……………………………………………………………..

2. Khuyết điểm

…………….………..……….……………………………………………………………..

…………….………..……….……………………………………………………………..

…………….………..……….…………………………………………………………….. …………….………..……….……………………………………………………………. …………….………..……….……………………………………………………………..

…………….………..……….…………………………………………………………….

3. Đề nghị cho bảo vệ hay không ?............................................................................................

4. Đánh giá loại : ………………………………………………………………………………

5. Điểm : ………………………………………………………………………………………

Tp. Hồ Chí Minh, ngày tháng 12 năm 2023

Giáo viên phản biện

(Ký & ghi rõ họ tên)

MỤC LỤC

LỜI CẢM ƠN ........................................................................................................................1

DANH MỤC BẢNG BIỂU ...................................................................................................2

- Vẫn trích xuất được nhưng hiệu quả nhận thấy không cao so với Rerank

### Bước 5: Generation

- Ở đây nhóm tiến hành lựa chọn LLama3 và Gemini để sử dụng Model. Mô hình được gọi từ API để có thể tối ưu hóa thời gian trả về cho người dùng- nhanh hơn so với chạy Local

In [86]:
text = ''.join([x1.page_content for x1 in compressed_docs])

In [87]:
print(text)

3.7745 xuống còn 0.1936. Các biến động nay có thể la do sự đa dạng của dữ liệu.

Mất mát đao tạo giảm một cách đáng kể từ 3.7745 xuống 0.1936 qua 1401 bước.

Sự giảm mất mát nay có vẻ tích cực va thường la dấu hiệu của việc mô hình đang

học va tinh chỉnh tham số hiệu quả, hoặc sự nhạy cảm của mô hình với các mẫu

cụ thể.

Bước thứ 1101 có mất mát đao tạo lớn la 1.1331. Điều nay có thể la do mô hình

gặp phải một trường hợp khó hoặc không phổ biến trong dữ liệu, hoặc có thể la

do sự nhạy cảm của mô hình đối với một số mẫu cụ thể.

Trang | 72

CHƯƠNG 5. XÂY DỰNG ỨNG DỤNG DEMO

5.1. Phân tích, đặc tả yêu cầu

5.1.1. Mô tả chung

Sản phẩm cung cấp một giao diện trực quan cho người dùng, cho phép

họ thuận tiện ghi âm va nhập văn bản trực tiếp trên trình duyệt web(ở đây trên

nền tảng Streamlit). Ứng dụng chatbot được thiết kế với mục đích cung cấp

một giao diện thân thiện, cho phép người dùng đặt câu hỏi một cách dễ dang

bằng cả văn bản va giọng nói, va nhận câu trả lời ngay lập tức.



In [ ]:
# Thiết kế Prompt hiệu quả

In [88]:
prompt1 = f"Dựa vào nội dung trả lời dưới đây:{text}.Hãy trả lời câu hỏi sau đây: Hãy cho biết tên giáo viên phản biện trong demo1.pdf"

In [89]:

llm_response = question_answer(prompt1)

In [90]:
print(llm_response)

Tên giáo viên phản biện trong demo1.pdf là TS. Nguyễn Thanh Sơn.


- Nhận xét: 
